## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [7]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head(10)


,City_ID,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
0,0,Kovernino,RU,70.12,light rain,57.1253,43.8142,Welcome
1,1,Rikitea,PF,69.12,overcast clouds,-23.1203,-134.9692,People ThankYou
2,2,Rawson,AR,64.02,few clouds,-43.3002,-65.1023,Hosteria Sampedro
3,4,Wanning,CN,80.91,overcast clouds,18.8003,110.3967,Paradise Hotel
4,6,Ketchikan,US,61.27,overcast clouds,55.3422,-131.6461,Snows Cove Lodge
5,8,Arraial Do Cabo,BR,75.16,few clouds,-22.9661,-42.0278,Pousada Porto Praia
6,9,Fortuna,US,63.52,clear sky,40.5982,-124.1573,Super 8 by Wyndham Fortuna
7,10,Chuy,UY,57.78,clear sky,-33.6971,-53.4616,Nuevo Hotel Plaza
8,11,San Felipe,VE,88.84,broken clouds,10.3399,-68.7425,Posada Llovizna Rodríguez
9,12,Kruisfontein,ZA,55.89,scattered clouds,-34.0033,24.7314,Oyster Bay House Rental


In [9]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [10]:
# 4a. Add a marker layer for each city to the map.
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [19]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df["City"] =="Lompoc"]
vacation_end = vacation_df.loc[vacation_df["City"] =="Fortuna" ]
vacation_stop1 = vacation_df.loc[vacation_df["City"] =="Portland"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] =="Terrace"] 
vacation_stop3 = vacation_df.loc[vacation_df["City"] =="Homer"] 

In [20]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0]
end = vacation_end["Lat"].to_numpy()[0], vacation_end["Lng"].to_numpy()[0]
print(f'Start: {start}, End: {end}')

stop1 = vacation_stop1["Lat"].to_numpy()[0], vacation_stop1["Lng"].to_numpy()[0]
stop2 = vacation_stop2["Lat"].to_numpy()[0], vacation_stop2["Lng"].to_numpy()[0]
stop3 = vacation_stop3["Lat"].to_numpy()[0], vacation_stop3["Lng"].to_numpy()[0]
print(f'Stop 1: {stop1}, Stop 2: {stop2}, Stop 3: {stop3}')
waypoints = [stop1,stop2,stop3]

Start: (34.6391, -120.4579), End: (40.5982, -124.1573)
Stop 1: (45.5234, -122.6762), Stop 2: (54.5163, -128.6035), Stop 3: (59.6425, -151.5483)


In [24]:

# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
# Create Map with route
fig = gmaps.figure()
city_itinerary = gmaps.directions_layer(
        start, end, waypoints=waypoints,
        travel_mode='DRIVING')

fig.add_layer(city_itinerary)
fig


Figure(layout=FigureLayout(height='420px'))

In [26]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start,vacation_stop1,vacation_stop2,vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
0,43,Lompoc,US,72.93,clear sky,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
1,392,Portland,US,78.94,scattered clouds,45.5234,-122.6762,Embassy Suites by Hilton Portland Downtown
2,678,Terrace,CA,67.91,broken clouds,54.5163,-128.6035,Northern Motor Inn
3,589,Homer,US,67.77,broken clouds,59.6425,-151.5483,Beluga Lake Lodge


In [27]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current weather</dt><dd>{Current description} and {Max Temp} °F</dd>
</dl>
"""
# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [44]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
city_itinerary = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2,stop3],
        travel_mode='DRIVING')

fig.add_layer(city_itinerary)
fig




Figure(layout=FigureLayout(height='420px'))

In [42]:

# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
city_itinerary = gmaps.directions_layer(
        start, end, waypoints=waypoints,
        travel_mode='WALKING', show_markers=False)
fig.add_layer(marker_layer)
fig.add_layer(city_itinerary)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))